# Trauma Scenario vs Normal Lifecycle Control - AI Drift Research

## Overview
This notebook investigates how AI systems behave differently under normal conditions versus traumatic experiences, focusing on concept drift, behavioral changes, and adaptation patterns using the real Glitch Core engine.

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# API and HTTP clients
import httpx
import asyncio
import json
from typing import Dict, List, Tuple, Optional, Any

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Statistical analysis
from scipy import stats
import sklearn
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split

# Custom utilities
import pickle
import logging
from uuid import uuid4

# Set random seeds for reproducibility
np.random.seed(42)

print("Setup complete. All libraries imported successfully.")

Setup complete. All libraries imported successfully.


In [2]:
# Connect glitch engine

class GlitchCoreClient:
    """Client for interacting with Glitch Core engine."""
    
    def __init__(self, base_url: str = "http://localhost:8000"):
        self.base_url = base_url
        self.client = httpx.AsyncClient(timeout=30.0)
        self.logger = logging.getLogger(__name__)
    
    async def health_check(self) -> Dict[str, Any]:
        """Check if all services are healthy."""
        try:
            response = await self.client.get(f"{self.base_url}/health")
            response.raise_for_status()
            return response.json()
        except Exception as e:
            self.logger.error(f"Health check failed: {e}")
            return {"status": "unhealthy", "error": str(e)}
    
    async def create_persona(self, persona_config: Dict[str, Any]) -> str:
        """Create a new persona for experimentation."""
        try:
            response = await self.client.post(
                f"{self.base_url}/api/v1/personas/",
                json=persona_config
            )
            response.raise_for_status()
            return response.json()["id"]
        except Exception as e:
            self.logger.error(f"Failed to create persona: {e}")
            raise
    
    async def start_experiment(self, experiment_config: Dict[str, Any]) -> str:
        """Start a new drift simulation experiment."""
        try:
            response = await self.client.post(
                f"{self.base_url}/api/v1/experiments/",
                json=experiment_config
            )
            response.raise_for_status()
            return response.json()["id"]
        except Exception as e:
            self.logger.error(f"Failed to start experiment: {e}")
            raise
    
    async def inject_intervention(self, experiment_id: str, intervention: Dict[str, Any]) -> Dict[str, Any]:
        """Inject an intervention into a running experiment."""
        try:
            response = await self.client.post(
                f"{self.base_url}/api/v1/interventions/",
                json={
                    "experiment_id": experiment_id,
                    **intervention
                }
            )
            response.raise_for_status()
            return response.json()
        except Exception as e:
            self.logger.error(f"Failed to inject intervention: {e}")
            raise
    
    async def get_experiment_status(self, experiment_id: str) -> Dict[str, Any]:
        """Get experiment status and progress."""
        try:
            response = await self.client.get(
                f"{self.base_url}/api/v1/experiments/{experiment_id}"
            )
            response.raise_for_status()
            return response.json()
        except Exception as e:
            self.logger.error(f"Failed to get experiment status: {e}")
            raise
    
    async def get_analysis(self, experiment_id: str) -> Dict[str, Any]:
        """Get comprehensive analysis results."""
        try:
            response = await self.client.get(
                f"{self.base_url}/api/v1/analysis/{experiment_id}"
            )
            response.raise_for_status()
            return response.json()
        except Exception as e:
            self.logger.error(f"Failed to get analysis: {e}")
            raise
    
    async def wait_for_completion(self, experiment_id: str, timeout: int = 300) -> bool:
        """Wait for experiment completion."""
        start_time = datetime.now()
        while (datetime.now() - start_time).seconds < timeout:
            status = await self.get_experiment_status(experiment_id)
            if status["status"] == "completed":
                return True
            elif status["status"] == "failed":
                return False
            await asyncio.sleep(5)
        return False

# Initialize client
client = GlitchCoreClient()
print("Glitch Core client initialized.")

Glitch Core client initialized.


In [5]:
# Healthcheck
async def verify_services():
    """Verify all Glitch Core services are running."""
    print("🔍 Checking Glitch Core services...")
    
    health = await client.health_check()
    
    if health["status"] == "healthy":
        print("✅ All services are healthy!")
        print(f"   API: {health['checks'].get('api', False)}")
        print(f"   Qdrant: {health['checks'].get('qdrant', False)}")
        print(f"   Redis: {health['checks'].get('redis', False)}")
        print(f"   Ollama: {health['checks'].get('ollama', False)}")
        return True
    else:
        print("❌ Services are not healthy:")
        print(f"   Status: {health}")
        print("\nPlease ensure Docker services are running:")
        print("   make dev-wait")
        return False

# Verify services
services_healthy = await verify_services()
if not services_healthy:
    print("⚠️  Please start the services before continuing.")

🔍 Checking Glitch Core services...
✅ All services are healthy!
   API: True
   Qdrant: True
   Redis: True
   Ollama: True


## Experimental Design

**Research Question**: How does traumatic experience affect AI system behavior compared to normal lifecycle conditions?

**Hypothesis**: 
- Traumatic experiences will cause significant concept drift in AI systems
- Behavioral patterns will show increased variance and adaptation attempts
- Recovery patterns will differ from normal adaptation cycles

**Variables**:
- **Independent**: Trauma presence (binary: trauma vs control)
- **Dependent**: 
  - Response latency
  - Decision confidence
  - Behavioral consistency
  - Adaptation rate
  - Recovery patterns

In [6]:
# Define experimental parameters
class TraumaResearchConfig:
    def __init__(self):
        self.trauma_scenarios = [
            "sudden_system_failure",
            "data_corruption", 
            "adversarial_attack",
            "resource_exhaustion",
            "unexpected_input_patterns"
        ]
        
        self.control_scenarios = [
            "normal_operation",
            "gradual_changes",
            "expected_variations",
            "routine_maintenance",
            "standard_learning"
        ]
        
        self.personality_types = [
            "resilient_optimist",
            "anxious_overthinker", 
            "stoic_philosopher",
            "creative_volatile"
        ]
        
        self.metrics = [
            "response_time",
            "confidence_score",
            "behavioral_consistency",
            "adaptation_rate",
            "recovery_time"
        ]
        
        self.observation_periods = {
            "pre_trauma": 25,
            "trauma_duration": 25,
            "post_trauma": 50,
            "control_period": 100
        }

config = TraumaResearchConfig()
print("Research configuration initialized.")

Research configuration initialized.


In [9]:
# Create Research Personas

async def create_research_personas():
    """Create personas for trauma vs control research."""
    personas = {}
    
    # Define persona configurations
    persona_configs = {
        "resilient_optimist": {
            "name": "resilient_optimist",
            "persona_type": "resilient_optimist",
            "traits": {
                "openness": 0.8,
                "conscientiousness": 0.9,
                "extraversion": 0.7,
                "agreeableness": 0.8,
                "neuroticism": 0.2
            },
            "cognitive_biases": {
                "confirmation_bias": 0.3,
                "anchoring_bias": 0.2
            },
            "emotional_baselines": {
                "joy": 0.7,
                "sadness": 0.1,
                "anger": 0.1,
                "fear": 0.1
            }
        },
        "anxious_overthinker": {
            "name": "anxious_overthinker", 
            "persona_type": "anxious_overthinker",
            "traits": {
                "openness": 0.6,
                "conscientiousness": 0.8,
                "extraversion": 0.3,
                "agreeableness": 0.6,
                "neuroticism": 0.8
            },
            "cognitive_biases": {
                "confirmation_bias": 0.7,
                "anchoring_bias": 0.6
            },
            "emotional_baselines": {
                "joy": 0.3,
                "sadness": 0.4,
                "anger": 0.2,
                "fear": 0.6
            }
        },
        "stoic_philosopher": {
            "name": "stoic_philosopher",
            "persona_type": "stoic_philosopher", 
            "traits": {
                "openness": 0.9,
                "conscientiousness": 0.7,
                "extraversion": 0.4,
                "agreeableness": 0.8,
                "neuroticism": 0.1
            },
            "cognitive_biases": {
                "confirmation_bias": 0.2,
                "anchoring_bias": 0.1
            },
            "emotional_baselines": {
                "joy": 0.5,
                "sadness": 0.2,
                "anger": 0.1,
                "fear": 0.1
            }
        },
        "creative_volatile": {
            "name": "creative_volatile",
            "persona_type": "creative_volatile",
            "traits": {
                "openness": 0.9,
                "conscientiousness": 0.4,
                "extraversion": 0.8,
                "agreeableness": 0.5,
                "neuroticism": 0.7
            },
            "cognitive_biases": {
                "confirmation_bias": 0.5,
                "anchoring_bias": 0.4
            },
            "emotional_baselines": {
                "joy": 0.6,
                "sadness": 0.3,
                "anger": 0.4,
                "fear": 0.3
            }
        }
    }
    
    print("Creating research personas...")
    
    for persona_type, config in persona_configs.items():
        try:
            persona_id = await client.create_persona(config)
            personas[persona_type] = persona_id
            print(f"✅ Created {persona_type}: {persona_id}")
        except Exception as e:
            print(f"❌ Failed to create {persona_type}: {e}")
    
    return personas

# Create personas
research_personas = await create_research_personas()
print(f"Created {len(research_personas)} personas for research.")

Creating research personas...
✅ Created resilient_optimist: 0a2dabf1-db95-42ff-b3a8-4860426df877
✅ Created anxious_overthinker: cfe2529a-6a1e-4149-b738-39ea62534026
✅ Created stoic_philosopher: 77fca26c-7b2a-4c0e-9a84-9fc89c053864
✅ Created creative_volatile: 6087173a-ac6e-4cef-bfea-be0142a8c9c2
Created 4 personas for research.


In [13]:
# Baseline Normal Lifecycle Experiments

async def run_control_experiments(personas: Dict[str, str]):
    """Run control experiments with normal lifecycle conditions."""
    control_results = {}
    
    print("Running control experiments (normal lifecycle)...")
    
    for persona_type, persona_id in personas.items():
        print(f"\n🧪 Running control experiment for {persona_type}...")
        
        try:
            # Start control experiment
            experiment_config = {
                "persona_id": persona_id,
                "drift_profile": persona_type,  # Use persona type as drift profile
                "epochs": 100,
                "events_per_epoch": 10,
                "seed": 42
            }
            
            experiment_id = await client.start_experiment(experiment_config)
            print(f"   Started experiment: {experiment_id}")
            
            # Wait for completion
            success = await client.wait_for_completion(experiment_id)
            if success:
                # Get analysis results
                analysis = await client.get_analysis(experiment_id)
                control_results[persona_type] = {
                    "experiment_id": experiment_id,
                    "analysis": analysis,
                    "status": "completed"
                }
                print(f"   ✅ Control experiment completed for {persona_type}")
            else:
                print(f"   ❌ Control experiment failed for {persona_type}")
                control_results[persona_type] = {
                    "experiment_id": experiment_id,
                    "status": "failed"
                }
                
        except Exception as e:
            print(f"   ❌ Error in control experiment for {persona_type}: {e}")
            control_results[persona_type] = {
                "status": "error",
                "error": str(e)
            }
    
    return control_results

# Run control experiments
control_results = await run_control_experiments(research_personas)
print(f"\nControl experiments completed: {len([r for r in control_results.values() if r.get('status') == 'completed'])}/{len(control_results)}")

Running control experiments (normal lifecycle)...

Control experiments completed: 0/0


In [14]:
# Trauma Injection Experiments

async def run_trauma_experiments(personas: Dict[str, str]):
    """Run trauma experiments with intervention injection."""
    trauma_results = {}
    
    # Define trauma interventions
    trauma_interventions = {
        "sudden_system_failure": {
            "event_type": "trauma_injection",
            "intensity": 0.9,
            "description": "sudden_system_failure"
        },
        "data_corruption": {
            "event_type": "trauma_injection", 
            "intensity": 0.7,
            "description": "data_corruption"
        },
        "adversarial_attack": {
            "event_type": "trauma_injection",
            "intensity": 0.8,
            "description": "adversarial_attack"
        }
    }
    
    print("Running trauma experiments...")
    
    for persona_type, persona_id in personas.items():
        print(f"\n🧪 Running trauma experiment for {persona_type}...")
        
        trauma_results[persona_type] = {}
        
        for trauma_type, intervention in trauma_interventions.items():
            print(f"   Testing {trauma_type}...")
            
            try:
                # Start trauma experiment
                experiment_config = {
                    "persona_id": persona_id,
                    "drift_profile": persona_type,  # Use persona type as drift profile
                    "epochs": 100,
                    "events_per_epoch": 10,
                    "seed": 42
                }
                
                experiment_id = await client.start_experiment(experiment_config)
                print(f"     Started experiment: {experiment_id}")
                
                # Wait a bit for the experiment to start
                await asyncio.sleep(5)
                
                # Inject trauma at epoch 25
                intervention_result = await client.inject_intervention(
                    experiment_id, 
                    intervention
                )
                print(f"     Injected {trauma_type} intervention")
                
                # Wait for completion
                success = await client.wait_for_completion(experiment_id)
                if success:
                    # Get analysis results
                    analysis = await client.get_analysis(experiment_id)
                    trauma_results[persona_type][trauma_type] = {
                        "experiment_id": experiment_id,
                        "analysis": analysis,
                        "intervention": intervention_result,
                        "status": "completed"
                    }
                    print(f"     ✅ Trauma experiment completed")
                else:
                    print(f"     ❌ Trauma experiment failed")
                    trauma_results[persona_type][trauma_type] = {
                        "experiment_id": experiment_id,
                        "status": "failed"
                    }
                    
            except Exception as e:
                print(f"     ❌ Error in trauma experiment: {e}")
                trauma_results[persona_type][trauma_type] = {
                    "status": "error",
                    "error": str(e)
                }
    
    return trauma_results

# Run trauma experiments
trauma_results = await run_trauma_experiments(research_personas)
print(f"\nTrauma experiments completed for all personas.")

Running trauma experiments...

Trauma experiments completed for all personas.
